In [167]:
# ### Selenium 설치
# !pip install selenium

# ### lxml 설치
# # !python -m pip install --upgrade pip
# !pip install lxml

# ## 이 부분은 Colab에서 chromdriver 사용 시 오류나는 경우 설치
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

# ## Error 발생 시:  Message: Service chromedriver unexpectedly exited. Status code was: 1
# !pip install webdriver_manager

In [1]:
import pandas as pd
import time

import requests
from bs4 import BeautifulSoup
from urllib import parse

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# **Browser Setting**

In [101]:
### Chrome web 사용.

options = webdriver.ChromeOptions()                     # Chrome 창의 옵션을 설정할 수 있는 코드.
options.add_argument('--no-sandbox')                    
options.add_argument('--disable-dev-shm-usage')         
options.add_argument('--headless')                      # headless : Broswer 창을 띄우지 않고 수행 (colab에서는 필수).
# options.add_argument("window-size = 1920,1080")       # window size 설정


browser = webdriver.Chrome(options = options)           # 위 Option을 적용한 Chrome 창을 실행.
browser.maximize_window()                               # browser 창 최대화

# **특정 Youtube 접속**

## **1. 검색과 클릭으로 접속**

In [102]:
# ### 페이지 이동
# url = "https://www.youtube.com"
# browser.get(url)

# ### 유튜버 검색
# elem = browser.find_element("name", "search_query")
# elem.send_keys("앰비션")
# elem.send_keys(Keys.ENTER)
# time.sleep(2)   # 검색 완료까지 대기. 

# #### 유튜브 채널 입장
# elem = browser.find_element("id", "main-link")
# elem.click()

# #### 유튜브 동영상 목록 
# # WebDriverWait(browser, n).until(): until에서 설정한 Event가 완료될때까지 n초 대기
# # EC.presence_of_element_located: Element 존재여부 확인
# # By.PARTIAL_LINK_TEXT: 보이는 텍스트에 검색 값이 포함된 앵커 요소를 찾으며, 여러 요소가 일치하는 경우 첫 번째 요소만 선택.
# try :
#     elem = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "동영상")))
#     elem.click()

# finally :  # 동작 완료 및 실패 시
#     pass

# time.sleep(2)

## **2. URL 직접 입력 접속**

In [103]:
url = "https://www.youtube.com/@AMTUBE/videos"
browser.get(url)

# **특정 유튜브의 동영상 댓글 긁어오기**

## **_동영상 URL 모으기**

In [104]:
soup = BeautifulSoup(browser.page_source, 'html.parser')
TL_list = soup.select("div#contents div#content a#video-title-link")

TL_df = pd.DataFrame(columns = ["Title", "Link"])
for tag_ in TL_list:
    title = tag_.text
    link = parse.urljoin(url, tag_.get("href"))
    temp = pd.DataFrame([title, link], index = ["Title", "Link"]).T
    TL_df = pd.concat([TL_df, temp], axis = 0)

TL_df.reset_index(drop = True, inplace = True)

In [105]:
TL_df.head(3)

,Title,Link
0,원딜 멘탈 케어하기 (난이도 : 극악),https://www.youtube.com/watch?v=RGJblo-J8Mk
1,'떨어지는 강찬밥과 고수달',https://www.youtube.com/watch?v=bTj5lO-fNZU
2,왜?.. 나한테만..? 왜?,https://www.youtube.com/watch?v=1iu0yu5Hqkw


## **_동영상 접속 후 댓글 긁어오기**

In [106]:
# 유튜브 동영상 들어가기

Comment_dict = {}
Comment_cnt = 0

for i in range(0, 3) :   # 3개의 동영상
    
    ######################## 유튜브 댓글 긁기위해 i번째 동영상 링크 접속 ########################
    browser.get(TL_df.loc[i, "Link"])
    print("=="*25)
    print("%d번째 동영상 댓글 크롤링 시작!" % (i+1))

    
    ######################## 댓글을 보기 위해 스크롤 내리기 ########################
    
    interval = 3    # 3초에 한번씩 스크롤 내림

    now_height = browser.execute_script("return document.documentElement.scrollHeight")     # now_height: 현재의 스크롤 높이로 사용  // # 네이버의 경우 documentElement 대신 body
    site_height = browser.execute_script("return document.documentElement.scrollHeight")    # site_height: 화면의 스크롤 높이로 저장

    time.sleep(5)
    browser.execute_script("window.scrollTo(0, 1000)")   # 한 번 내리고 댓글 로딩 대기.
    time.sleep(5)

    # 반복 수행
    while True :
        
        browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")   # 스크롤을 가장 아래로 내림

        time.sleep(interval)   # 페이지 로딩 대기

        # 현재 문서 높이를 가져와 저장
        curr_height = browser.execute_script("return document.documentElement.scrollHeight")
        if curr_height >= site_height * 3 :     # 스크롤 3번 내리면 정지
            break

        now_height = curr_height                # 과거 높이를 현재 높이로 업데이트

    print(" -스크롤 완료! | (현재 페이지 높이 : %d)" % (curr_height))
    

    ######################## ID와 댓글 긁어오기 ########################
    soup = BeautifulSoup(browser.page_source , "lxml")
    
    # 댓글 박스 가져오기
    comment_box = soup.select("ytd-item-section-renderer#sections > div#contents")[0]

    # 댓글 리스트 길이 가져오기
    comment_len = len(comment_box.select("ytd-comment-thread-renderer.style-scope.ytd-item-section-renderer"))
    Comment_cnt += comment_len
    print(" -수집한 누적 댓글 수: %d개" % (Comment_cnt))

    # 반복문 작업
    for idx in range(comment_len) :
        # 댓글 리스트 가져오기
        comments = comment_box.select("ytd-comment-thread-renderer.style-scope.ytd-item-section-renderer")

        name = comments[idx].select_one("a#author-text").text.strip()
        comment = comments[idx].select_one("yt-formatted-string#content-text").text.strip()
        try:
            like = comments[idx].select_one("span#vote-count-middle").attrs['aria-label']
        except :
            like = ""
        
        Comment_dict[name] = {"Like": like, "Comment": comment}

    print("%d번째 동영상 댓글 크롤링 끝!" % (i+1))

1번째 동영상 댓글 크롤링 시작!
 -스크롤 완료! | (현재 페이지 높이 : 5658)
 -수집한 누적 댓글 수: 40개
1번째 동영상 댓글 크롤링 끝!
2번째 동영상 댓글 크롤링 시작!
 -스크롤 완료! | (현재 페이지 높이 : 5734)
 -수집한 누적 댓글 수: 80개
2번째 동영상 댓글 크롤링 끝!
3번째 동영상 댓글 크롤링 시작!
 -스크롤 완료! | (현재 페이지 높이 : 5773)
 -수집한 누적 댓글 수: 120개
3번째 동영상 댓글 크롤링 끝!


In [107]:
### 브라우저 종료
browser.quit()

In [108]:
Comment_df = pd.DataFrame.from_dict(Comment_dict, orient = 'index')
Comment_df["ID"] = Comment_df.index
Comment_df = Comment_df[["ID", "Comment", "Like"]].reset_index(drop = True)
print(Comment_df.shape)
Comment_df.head(3)

(98, 3)


,ID,Comment,Like
0,@choi_yongminsu,️5월 4일️\nXiao 68승 55패[7승8패]\n ㄴM 126LP (-13LP)...,좋아요 91개
1,@SVTmcarat0526,이정도면 같은 팀인게 아닐까 싶을정도로 같은 운명이 되는 두 사람ㅋㅋㅋㅋㅋㅋㅋ,좋아요 59개
2,@user-xi7nh9ni4p,와 점수변동이 1000을 넘어 1300에 가까워지는 찬밥이형ㅋㅋㅋㅋㅋㅋ 드븐 정신병...,좋아요 18개


In [ ]:
# Comment_df.to_csv("Youtube_comment.csv", index = False)